In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))

<strong>Notebook run on: 2020-04-20 16:53 | by soge | Using ris library version: 1.4.7 | File Location: E:\RIS\Staff Folders\Samuel\Requests\AG\Bike_Volumes

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='bicycle')


Additional database connection details required:
User name (bicycle):soge
Password (bicycle)········
Database connection (PG) to bicycle on dotdevrhpgsql01 - user: soge 
Connection established 2020-04-20 16:53:57


### Formatting 'For Arthur' Excel Sheet

#### Reading in 'For Arthur' Sheet

In [3]:
bike_volumes = pd.read_excel('Bike Counts Linked to TIMS Raw Files- 2.26.20.xlsx', sheet_name='For Arthur')

#### Resetting Sheet Header

In [4]:
bike_volumes.columns = bike_volumes.iloc[1]
bike_volumes = bike_volumes[2:]

#### Limiting table to rows with (before weekday date, before weekday count, 1-yr after weekday date, 1-yr after weekday count)where after date is 1 year after installation 

In [5]:
bv = bike_volumes.loc[(bike_volumes['Before Weekend'].isnull()) & (bike_volumes['1 Year After (weekday)'].notnull()) & (bike_volumes['1 Year After (weekday)'] != 'Only have 3 years after') & (bike_volumes['Before (weekday)'].notnull())].copy()
bv = bv.loc[(bv['1 Year After (weekday)'].dt.year - bv['Before (weekday)'].dt.year) < 4]
bv = bv.reset_index()

In [6]:
bv_id1 = (tuple(list(bv.loc[bv.BikeID.notnull()].BikeID)))

### Formatting '#### Reading in 'Cleaned Combined Bike Volumes' Excel Sheet

#### Reading in 'Cleaned Combined Bike Volumes' Sheet

In [7]:
bike_volumes2 = pd.read_excel('cleaned_combined_bike_volumes_03022020.xlsx', sheet_name='bikes 03032020')

In [8]:
bv2 = bike_volumes2.loc[bike_volumes2['Before (weekday)'].notnull()].copy()
bv2 = bv2.loc[(bv2['Before (weekday)'].notnull()) & (bv2['Before total'].notnull()) & (bv2['1 Year After (weekday)'].notnull()) & (bv2['1 Year After (weekday)'] != 'Only have 3 years after') & (bv2['1 year after total'].notnull())]

In [9]:
bv2 = bv2.loc[(bv2['1 Year After (weekday)'].dt.year - bv2['Before (weekday)'].dt.year)< 4]
bv2 = bv.reset_index()

In [10]:
bv_id2  = (tuple(map(int, list(bv2.loc[bv2.BikeID.notnull()].BikeID))))

In [11]:
x = tuple(set(bv_id1 + bv_id2))

### All Bike id Counts that are not in 'For Arthure' Excel Sheet nor in 'Cleaned Combined Bike Volumes' Excel Sheet

In [12]:
bvm = db.dfquery("""

SELECT DISTINCT bikeid, facilitycl, street, fromstreet, tostreet, 
     instdate, moddate, date_part('year', moddate) yr
FROM bike_network_2019_11_22
WHERE date_part('year', moddate) between 2009 and 2016
and bikeid not in {}
ORDER BY yr


""".format(tuple(x))
)

In [13]:
bvm.to_excel('bike_volumes_missing_{}.xlsx'.format(ts),index = 0)

In [14]:
bv= bv.drop(columns = ['index'])
bv2 = bv2.drop(columns=['level_0', 'index'])

In [15]:
bike_volumes = pd.concat([bv,bv2])
bike_volumes_unique = bike_volumes.drop_duplicates('BikeID',keep='first').reset_index()
bids = tuple(map(int,list(bike_volumes_unique.fillna(0).BikeID)))

In [16]:
db.connect()

Database connection (PG) to bicycle on dotdevrhpgsql01 - user: soge 
Connection established 2020-04-20 16:54:00


In [17]:
bids_from_to = db.dfquery("""

SELECT DISTINCT bikeid, street, fromstreet, tostreet 
FROM bike_network_2019_11_22
WHERE date_part('year', moddate) between 2009 and 2016
AND bikeid in {}
ORDER BY bikeid

""".format(bids)
)

In [18]:
writer = pd.ExcelWriter('bike_volumes_{}.xlsx'.format(ts), engine='xlsxwriter')    
bike_volumes.to_excel(writer, sheet_name='Concatenated', index = False)
bids_from_to.to_excel(writer, sheet_name='Unique', index = False)
#bv.to_excel(writer, sheet_name='For Arthur', index = False)
#bv2.to_excel(writer, sheet_name='bikes 03032020', index = False)
writer.save() 